# Introduction

Hello, world! Thanks for trying to understand how to use the [Django Stomp](https://github.com/juntossomosmais/django-stomp) library! We'll follow some examples that will try to bring to you Django Stomp works and how you can use it for asynchronous communication!

Let's go!

## First things first

First of all, our demonstration here will use the [RabbitMQ](https://www.rabbitmq.com/) as its broker and the [STOMP](https://stomp.github.io/) procotol for its communication. We have setup everything on our [repository](https://github.com/fabiohk/django-stomp-examples) so that you can easily start this tutorial following its [README](https://github.com/fabiohk/django-stomp-examples/blob/main/README.md), but if you're not following it for some reason, try to setup it first before you run any code inside this notebook.

## And, it begins

If you have everything setup, try to login to the RabbitMQ Dashboard that is listening on port 15672 from your local host.

![RabbitMQ Dashboard](images/rabbitmq-dash.png)

Use the following credentials:

```text
username: guest
password: guest
```

And you are on!

![RabbitMQ Logged Dashboard](images/rabbitmq-logged-dash.png)

## Publishing the first message

We can now start our first interaction with Django-stomp. First, we'll need to create a publisher with a unique name (that is passed as parameter):

In [1]:
import uuid
from django_stomp.builder import build_publisher

publisher = build_publisher(f"publisher-{uuid.uuid4()}")

INFO     [2021-02-19 21:24:58,727] [MainThread] django_stomp: Server details connection: {'host': 'localhost', 'port': 61613, 'hostStandby': None, 'portStandby': None, 'outgoingHeartbeat': 10000, 'incomingHeartbeat': 10000, 'subscriptionId': None, 'vhost': None}. Extra params: {'use_ssl': False, 'client_id': 'publisher-ac2dc810-ef40-4093-9b2f-bafdc1fb8a02'}
INFO     [2021-02-19 21:24:58,728] [MainThread] django_stomp: Building publisher...
INFO     [2021-02-19 21:24:58,730] [MainThread] django_stomp: Use SSL? False. Version: 2


At this point, we just created a publisher, without any interaction with the broker. As we can see in the `Connection` tab

![No connections](images/no-connections.png)

Now, we can publish our first message! Let's try to publish to a simple queue named as `/queue/xpto`. For that, we'll need to use it inside the `auto_open_close_connection` context that, as it says, will open the connection and close it after the context is closed.

In [2]:
import uuid
from django_stomp.builder import build_publisher

publisher = build_publisher(f"publisher-{uuid.uuid4()}")

destination_name = "/queue/xpto"
message_to_publish = {
    "key": "value"
}
with publisher.auto_open_close_connection():
    publisher.send(message_to_publish, destination_name)

INFO     [2021-02-19 21:24:58,751] [MainThread] django_stomp: Server details connection: {'host': 'localhost', 'port': 61613, 'hostStandby': None, 'portStandby': None, 'outgoingHeartbeat': 10000, 'incomingHeartbeat': 10000, 'subscriptionId': None, 'vhost': None}. Extra params: {'use_ssl': False, 'client_id': 'publisher-3ed72d81-cef4-440d-88de-f6ad74a4f8c3'}
INFO     [2021-02-19 21:24:58,753] [MainThread] django_stomp: Building publisher...
INFO     [2021-02-19 21:24:58,754] [MainThread] django_stomp: Use SSL? False. Version: 2
INFO     [2021-02-19 21:24:58,755] [MainThread] django_stomp: It is not open. Starting...
INFO     [2021-02-19 21:24:59,259] [MainThread] stomp.py: attempt reconnection (True, None, 0)
INFO     [2021-02-19 21:24:59,261] [MainThread] stomp.py: Attempting connection to host localhost, port 61613
INFO     [2021-02-19 21:24:59,263] [MainThread] stomp.py: Established connection to host localhost, port 61613
INFO     [2021-02-19 21:24:59,271] [Thread-4] stomp.py: Start

We got it!

![Queue received the message](images/queue-received.png)
![Message received](images/received-message.png)

## Publishing under transaction

Imagine now that you have to publish a lot of messages to a queue and it **must** send it all or nothing. We can do that under a [STOMP transaction]()! To use it, with Django Stomp, we need to the context `do_inside_transaction`

In [3]:
import uuid
from django_stomp.builder import build_publisher
from django_stomp.services.producer import auto_open_close_connection
from django_stomp.services.producer import do_inside_transaction

publisher = build_publisher(f"publisher-{uuid.uuid4()}")

message_count = 5
destination_name = "/queue/calopsita"
message_to_publish = {
    "key": "value"
}


with auto_open_close_connection(publisher), do_inside_transaction(publisher):
    for _ in range(message_count):
        publisher.send(message_to_publish, destination_name)

INFO     [2021-02-19 21:24:59,952] [MainThread] django_stomp: Server details connection: {'host': 'localhost', 'port': 61613, 'hostStandby': None, 'portStandby': None, 'outgoingHeartbeat': 10000, 'incomingHeartbeat': 10000, 'subscriptionId': None, 'vhost': None}. Extra params: {'use_ssl': False, 'client_id': 'publisher-dbc1c07a-71c8-400b-92e4-b1c5f3a489a8'}
INFO     [2021-02-19 21:24:59,956] [MainThread] django_stomp: Building publisher...
INFO     [2021-02-19 21:24:59,958] [MainThread] django_stomp: Use SSL? False. Version: 2
INFO     [2021-02-19 21:25:00,466] [MainThread] stomp.py: attempt reconnection (True, None, 0)
INFO     [2021-02-19 21:25:00,470] [MainThread] stomp.py: Attempting connection to host localhost, port 61613
INFO     [2021-02-19 21:25:00,473] [MainThread] stomp.py: Established connection to host localhost, port 61613
INFO     [2021-02-19 21:25:00,475] [Thread-5] stomp.py: Starting receiver loop (<Thread(Thread-5, started daemon 123145549148160)>)
INFO     [2021-02-1

Again, everything went smooth!

![Transaction](images/transaction.png)

## How to listen to those messages?

If you install Django Stomp within your Django App, you'll have a nice Django command that will allow you to set a callback to process the messages from a queue.

Your callback **must** have the following signature:

```python
from django_stomp.services.consumer import Payload

def callback(payload: Payload):
    ...
```

In the `Payload`, you'll have:

```python
class Payload:
    ack: Callable # To acknowledge
    nack: Callable # To not acknowledge
    headers: Dict # The message headers
    body: Dict # The message body
```

Now, suppose that your queue name is `/queue/xpto` and your callback is inside the module `callback.method`. To start your consumer, you have to run:

```sh
python manage.py pubsub /queue/xpto callback.method
```

For example only, we will use the following callback:

```python
import logging

from django_stomp.services.consumer import Payload

logger = logging.getLogger(__name__)

def ack_callback(payload: Payload):
    """
    Acknoweledge that the message was received, informing the broker that it can
    remove the message from queue.
    """
    logger.info(payload)
    payload.ack()
```

And, start the consumer inside Python code:

In [4]:
from django.core.management import call_command
from concurrent.futures import ThreadPoolExecutor

pool_executor = ThreadPoolExecutor()

callback_str = "examples.examples.ack_callback"
queue_name = "/queue/xpto"

pool_executor.submit(call_command, "pubsub", queue_name, callback_str)

<Future at 0x1050a51f0 state=running>

And the message was consumed!

![Message consumed](images/message-consumed.png)

## Virtual Topics?

What if you want to publish the same message to multiple queues? [Virtual Topics](https://activemq.apache.org/virtual-destinations.html) to the rescue! It's a concept we bring from ActiveMQ and it works as the following:

- Consume the message from `Consumer.app-name.VirtualTopic.topic-name` (it **must** follow this pattern `Consumer.*.VirtualTopic.*`)
- Publish a message to `/topic/VirtualTopic.topic-name` (it **must** follow this initial pattern `/topic/VirtualTopic.*`)

![Virtual Topics](images/virtual-topics.png)

It's important to mention that the queue from a consumer will only start to receive published messages after it's initial setup (after a consumer is setup). Any message published before won't be received.

Let's try to setup 2 consumers from the topic `/topic/VirtualTopic.xpto`

In [5]:
from django.core.management import call_command
from concurrent.futures import ThreadPoolExecutor

pool_executor = ThreadPoolExecutor()

callback_str = "examples.examples.ack_callback"
queue_name_from_consumer_A = "Consumer.app-A.VirtualTopic.xpto"
queue_name_from_consumer_B = "Consumer.app-B.VirtualTopic.xpto"

pool_executor.submit(call_command, "pubsub", queue_name_from_consumer_A, callback_str)
pool_executor.submit(call_command, "pubsub", queue_name_from_consumer_B, callback_str)

<Future at 0x1050c3430 state=running>

And, we have the two new queues

![Virtual Topic Consumers](images/topic-consumers.png)
![Consumer A](images/consumer-a.png)
![Consumer B](images/consumer-b.png)

And, we can now send a message to `/topic/VirtualTopic.xpto` and the two consumers will receive the same message!

In [6]:
import uuid
from django_stomp.builder import build_publisher

publisher = build_publisher(f"publisher-{uuid.uuid4()}")

destination_name = "/topic/VirtualTopic.xpto"
message_to_publish = {
    "key": "value"
}
with publisher.auto_open_close_connection():
    publisher.send(message_to_publish, destination_name)

INFO     [2021-02-19 21:25:01,199] [MainThread] django_stomp: Server details connection: {'host': 'localhost', 'port': 61613, 'hostStandby': None, 'portStandby': None, 'outgoingHeartbeat': 10000, 'incomingHeartbeat': 10000, 'subscriptionId': None, 'vhost': None}. Extra params: {'use_ssl': False, 'client_id': 'publisher-90af70f8-cbf9-4f1a-b152-c1f5cdd6cdd2'}
INFO     [2021-02-19 21:25:01,212] [MainThread] django_stomp: Building publisher...
INFO     [2021-02-19 21:25:01,213] [MainThread] django_stomp: Use SSL? False. Version: 2
INFO     [2021-02-19 21:25:01,213] [MainThread] django_stomp: It is not open. Starting...
INFO     [2021-02-19 21:25:01,219] [ThreadPoolExecutor-1_0] django_stomp: Server details connection: {'host': 'localhost', 'port': 61613, 'hostStandby': None, 'portStandby': None, 'outgoingHeartbeat': 10000, 'incomingHeartbeat': 10000, 'subscriptionId': None, 'vhost': None}. Extra params: {'use_ssl': False, 'client_id': None}
INFO     [2021-02-19 21:25:01,221] [ThreadPoolExe

Provided parameters: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'skip_checks': True, 'source_destination': '/queue/xpto', 'callback_function': 'examples.examples.ack_callback'}
Calling internal service to consume messages
Provided parameters: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'skip_checks': True, 'source_destination': 'Consumer.app-A.VirtualTopic.xpto', 'callback_function': 'examples.examples.ack_callback'}
Calling internal service to consume messages
Provided parameters: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'skip_checks': True, 'source_destination': 'Consumer.app-B.VirtualTopic.xpto', 'callback_function': 'examples.examples.ack_callback'}
Calling internal service to consume messages


INFO     [2021-02-19 21:25:01,390] [Thread-6] stomp.py: Received frame: 'CONNECTED', len(body)=0
INFO     [2021-02-19 21:25:01,427] [Thread-8] stomp.py: Received frame: 'CONNECTED', len(body)=0
INFO     [2021-02-19 21:25:01,428] [Thread-9] stomp.py: Starting heartbeat loop
INFO     [2021-02-19 21:25:01,429] [ThreadPoolExecutor-2_0] stomp.py: Sending frame: 'SUBSCRIBE'
INFO     [2021-02-19 21:25:01,448] [Thread-10] stomp.py: Starting heartbeat loop
INFO     [2021-02-19 21:25:01,449] [ThreadPoolExecutor-1_0] stomp.py: Sending frame: 'SUBSCRIBE'
INFO     [2021-02-19 21:25:01,458] [Thread-11] stomp.py: Starting heartbeat loop
INFO     [2021-02-19 21:25:01,459] [ThreadPoolExecutor-2_1] stomp.py: Sending frame: 'SUBSCRIBE'
INFO     [2021-02-19 21:25:01,477] [ThreadPoolExecutor-2_0] django_stomp: Connected
INFO     [2021-02-19 21:25:01,490] [ThreadPoolExecutor-1_0] django_stomp: Connected
INFO     [2021-02-19 21:25:01,513] [ThreadPoolExecutor-1_0] stomp.py: Sending frame: 'DISCONNECT'
INFO   

INFO     [2021-02-19 21:25:01,798] [ThreadPoolExecutor-2_1] stomp.py: Sending frame: 'STOMP'
INFO     [2021-02-19 21:25:01,808] [ThreadPoolExecutor-1_0] stomp.py: Sending frame: 'STOMP'
INFO     [2021-02-19 21:25:01,970] [Thread-14] stomp.py: Received frame: 'CONNECTED', len(body)=0
INFO     [2021-02-19 21:25:01,972] [Thread-16] stomp.py: Starting heartbeat loop
INFO     [2021-02-19 21:25:01,973] [ThreadPoolExecutor-2_1] stomp.py: Sending frame: 'SUBSCRIBE'
INFO     [2021-02-19 21:25:01,975] [Thread-12] stomp.py: Received frame: 'CONNECTED', len(body)=0
INFO     [2021-02-19 21:25:01,976] [Thread-13] stomp.py: Received frame: 'CONNECTED', len(body)=0
INFO     [2021-02-19 21:25:01,977] [ThreadPoolExecutor-2_1] django_stomp: Connected
INFO     [2021-02-19 21:25:01,980] [Thread-17] stomp.py: Starting heartbeat loop
INFO     [2021-02-19 21:25:01,980] [ThreadPoolExecutor-2_0] stomp.py: Sending frame: 'SUBSCRIBE'
INFO     [2021-02-19 21:25:01,982] [MainThread] django_stomp: Connected
INFO    

INFO     [2021-02-19 21:25:02,164] [ThreadPoolExecutor-2_1] django_stomp: Connected
INFO     [2021-02-19 21:25:02,190] [Thread-20] stomp.py: Received frame: 'CONNECTED', len(body)=0
INFO     [2021-02-19 21:25:02,196] [Thread-22] stomp.py: Starting heartbeat loop
INFO     [2021-02-19 21:25:02,197] [ThreadPoolExecutor-2_0] stomp.py: Sending frame: 'SUBSCRIBE'
INFO     [2021-02-19 21:25:02,202] [ThreadPoolExecutor-2_0] django_stomp: Connected
INFO     [2021-02-19 21:25:02,496] [MainThread] stomp.py: Sending frame: 'SEND'
INFO     [2021-02-19 21:25:02,505] [MainThread] stomp.py: Sending frame: 'DISCONNECT'
INFO     [2021-02-19 21:25:02,513] [Thread-20] stomp.py: Received frame: 'MESSAGE', len(body)=16
INFO     [2021-02-19 21:25:02,513] [Thread-19] stomp.py: Received frame: 'MESSAGE', len(body)=16
INFO     [2021-02-19 21:25:02,518] [Thread-20] django_stomp: Message ID: T_891f5b60-6105-4e2e-8587-e8e3020dc4ed-listener@@session-7wF2g6CBJiVicj1snQTVXw@@1
INFO     [2021-02-19 21:25:02,518] [Thre

![Consumer A process message](images/consumed-a.png)
![Consumer B process message](images/consumed-b.png)

## What happen if my callback call `nack` instead of `ack`?

That's a good question and it's better represented with an example! Let's say that your callback do the following:

```python
import logging

from django_stomp.services.consumer import Payload

logger = logging.getLogger(__name__)


def nack_callback(payload: Payload):
    logger.info(payload)
    payload.nack()
```

And time to see it working:

In [7]:
import uuid
from django.core.management import call_command
from concurrent.futures import ThreadPoolExecutor
from django_stomp.builder import build_publisher

publisher = build_publisher(f"publisher-{uuid.uuid4()}")

pool_executor = ThreadPoolExecutor()

callback_str = "examples.examples.nack_callback"
queue_name = "/queue/nack-queue"

pool_executor.submit(call_command, "pubsub", queue_name, callback_str)

message = {
    "key": "value"
}
headers = {
    "header-key": "header-value"
}

with build_publisher(f"publisher-{uuid.uuid4()}").auto_open_close_connection() as publisher:
    publisher.send(message, queue_name, headers=headers)

INFO     [2021-02-19 21:25:02,577] [24d27ce4-dbe6-4046-95d4-78fe6f3fa5e6-listener_0] examples.examples: Payload(ack=<function Listener.on_message.<locals>.ack_logic at 0x1051c28b0>, nack=<function Listener.on_message.<locals>.nack_logic at 0x1051c2160>, headers={'subscription': '24d27ce4-dbe6-4046-95d4-78fe6f3fa5e6-listener', 'destination': '/topic/VirtualTopic.xpto', 'message-id': 'T_24d27ce4-dbe6-4046-95d4-78fe6f3fa5e6-listener@@session-A_dUHpvW7VQRsRTQIkZFbg@@1', 'redelivered': 'false', 'x-dead-letter-routing-key': 'DLQ.VirtualTopic.xpto', 'x-dead-letter-exchange': '', 'tshoot-destination': '/topic/VirtualTopic.xpto', 'correlation-id': 'bc794a71-72e9-4508-8214-6ed286bdd74e', 'persistent': 'true', 'content-type': 'application/json;charset=utf-8', 'content-length': '16'}, body={'key': 'value'})
INFO     [2021-02-19 21:25:02,578] [891f5b60-6105-4e2e-8587-e8e3020dc4ed-listener_0] stomp.py: Sending frame: 'ACK'
INFO     [2021-02-19 21:25:02,586] [MainThread] django_stomp: Server details 

Provided parameters: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'skip_checks': True, 'source_destination': '/queue/nack-queue', 'callback_function': 'examples.examples.nack_callback'}
Calling internal service to consume messages


INFO     [2021-02-19 21:25:02,829] [Thread-25] stomp.py: Received frame: 'CONNECTED', len(body)=0
INFO     [2021-02-19 21:25:02,834] [Thread-26] stomp.py: Starting heartbeat loop
INFO     [2021-02-19 21:25:02,835] [ThreadPoolExecutor-3_0] stomp.py: Sending frame: 'SUBSCRIBE'
INFO     [2021-02-19 21:25:02,840] [ThreadPoolExecutor-3_0] django_stomp: Connected
INFO     [2021-02-19 21:25:03,145] [MainThread] stomp.py: attempt reconnection (True, None, 0)
INFO     [2021-02-19 21:25:03,148] [MainThread] stomp.py: Attempting connection to host localhost, port 61613
INFO     [2021-02-19 21:25:03,153] [MainThread] stomp.py: Established connection to host localhost, port 61613
INFO     [2021-02-19 21:25:03,155] [Thread-27] stomp.py: Starting receiver loop (<Thread(Thread-27, started daemon 123145817780224)>)
INFO     [2021-02-19 21:25:03,156] [MainThread] stomp.py: Created thread <Thread(Thread-27, started daemon 123145817780224)> using func <function default_create_thread at 0x105136e50>
INFO  

It was sent to a new queue named as `DLQ.nack-queue`! That behaviour allow us to analyze the received message and if we need, to process it later.

![NACK queue](images/nack.png)
![NACK DLQ queue](images/nack-dlq.png)
![NACK DLQ message](images/nack-message.png)

## What happen if my callback raise an exception?

Again, let's try to see it with an example:

```python
import logging

from django_stomp.services.consumer import Payload

logger = logging.getLogger(__name__)


def exception_callback(payload: Payload):
    logger.info(payload)
    raise Exception("Won't process!")
```

In [8]:
import uuid
from django.core.management import call_command
from concurrent.futures import ThreadPoolExecutor
from django_stomp.builder import build_publisher

publisher = build_publisher(f"publisher-{uuid.uuid4()}")

pool_executor = ThreadPoolExecutor()

callback_str = "examples.examples.exception_callback"
queue_name = "/queue/exception-queue"

pool_executor.submit(call_command, "pubsub", queue_name, callback_str)

message = {
    "key": "value"
}
headers = {
    "header-key": "header-value"
}

with build_publisher(f"publisher-{uuid.uuid4()}").auto_open_close_connection() as publisher:
    publisher.send(message, queue_name, headers=headers)

INFO     [2021-02-19 21:25:03,763] [MainThread] django_stomp: Server details connection: {'host': 'localhost', 'port': 61613, 'hostStandby': None, 'portStandby': None, 'outgoingHeartbeat': 10000, 'incomingHeartbeat': 10000, 'subscriptionId': None, 'vhost': None}. Extra params: {'use_ssl': False, 'client_id': 'publisher-57e6388d-38e1-4429-8ad9-c277c5c5295c'}
INFO     [2021-02-19 21:25:03,764] [MainThread] django_stomp: Building publisher...
INFO     [2021-02-19 21:25:03,765] [MainThread] django_stomp: Use SSL? False. Version: 2
INFO     [2021-02-19 21:25:03,770] [MainThread] django_stomp: Server details connection: {'host': 'localhost', 'port': 61613, 'hostStandby': None, 'portStandby': None, 'outgoingHeartbeat': 10000, 'incomingHeartbeat': 10000, 'subscriptionId': None, 'vhost': None}. Extra params: {'use_ssl': False, 'client_id': 'publisher-2ff83b6b-03b4-48d8-b526-dc4a92395f13'}
INFO     [2021-02-19 21:25:03,772] [MainThread] django_stomp: Building publisher...
INFO     [2021-02-19 21

Provided parameters: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'skip_checks': True, 'source_destination': '/queue/exception-queue', 'callback_function': 'examples.examples.exception_callback'}
Calling internal service to consume messages


INFO     [2021-02-19 21:25:03,980] [ThreadPoolExecutor-4_0] stomp.py: Created thread <Thread(Thread-30, started daemon 123145834569728)> using func <function default_create_thread at 0x105136e50>
INFO     [2021-02-19 21:25:04,007] [ThreadPoolExecutor-4_0] stomp.py: Sending frame: 'STOMP'
INFO     [2021-02-19 21:25:04,092] [Thread-30] stomp.py: Received frame: 'CONNECTED', len(body)=0
INFO     [2021-02-19 21:25:04,109] [Thread-31] stomp.py: Starting heartbeat loop
INFO     [2021-02-19 21:25:04,110] [ThreadPoolExecutor-4_0] stomp.py: Sending frame: 'SUBSCRIBE'
INFO     [2021-02-19 21:25:04,114] [ThreadPoolExecutor-4_0] django_stomp: Connected
INFO     [2021-02-19 21:25:04,278] [MainThread] stomp.py: attempt reconnection (True, None, 0)
INFO     [2021-02-19 21:25:04,281] [MainThread] stomp.py: Attempting connection to host localhost, port 61613
INFO     [2021-02-19 21:25:04,284] [MainThread] stomp.py: Established connection to host localhost, port 61613
INFO     [2021-02-19 21:25:04,287] 

Guess what?! Django Stomp will catch the exception and call `nack`, sending our message to the DLQ!

![Exception Queue](images/exception.png)
![Exception DLQ](images/exception-dlq.png)
![Exception Message](images/exception-message.png)
![Exception Logs](images/exception-log.png)

# The end

And that concludes our little tutorial on Django Stomp version 4.20.0! I hope that you could understand some of the features it has and if you have any questions or suggestions, feel free to create an issue [here](https://github.com/juntossomosmais/django-stomp/issues)!

Any contributions will be highly appreciated too!